# Area Summary

This is an exercise to see if performing individual summaries of areas in the home are useful to understand the overall home summary.

## Setup client

In [1]:
from secrets import get_secret
import hass_data
import hass_client
import openai
import model_client

MODEL_ID = "mistral-7b-inst.gguf"

secrets = get_secret("hostport")
client: hass_client.HomeAssistantClient = await hass_data.create_client(
    get_secret("hostport"), get_secret("hass_token")
)
openai = openai.OpenAI(base_url=get_secret("openai_url"), api_key=get_secret("openai_key"))
model = model_client.ModelClient(openai, MODEL_ID)

In [2]:
from hass_client.models import Area, Device, Entity, State
import hass_data

home: hass_data.Home = await hass_data.get_home(client)
home

Home(areas=[{'aliases': [], 'area_id': 'a9d99ac7b9664564affdc6e9e26dc57f', 'name': 'Living Room', 'picture': None}, {'aliases': [], 'area_id': 'b5c93f8f7b1a4d91af6099010c0709c7', 'name': 'Kitchen', 'picture': None}, {'aliases': [], 'area_id': 'cf60dac5b4054fb9be42f34158d2f1f7', 'name': 'Bedroom', 'picture': None}, {'aliases': [], 'area_id': '9a1e1d65616a4f4b83d0ddd6a1c5083e', 'name': 'Backyard', 'picture': None}, {'aliases': [], 'area_id': 'af09b663fb4d41e7a23cc58c58ba277c', 'name': 'Frontyard', 'picture': None}, {'aliases': [], 'area_id': '15de0e5dc35f4a86a66bcfd761983028', 'name': 'Dining Room', 'picture': None}, {'aliases': [], 'area_id': 'office', 'name': 'Office', 'picture': None}, {'aliases': [], 'area_id': 'garage', 'name': 'Garage', 'picture': None}, {'aliases': [], 'area_id': 'patio', 'name': 'Patio', 'picture': None}, {'aliases': [], 'area_id': 'family_room', 'name': 'Family Room', 'picture': None}, {'aliases': [], 'area_id': 'loft', 'name': 'Loft', 'picture': None}, {'aliase

In [3]:
d = home.devices_by_area('closet')
d[0]['id'], d[0]['name']

('548f06f0d7eb7b79de037861657e3e19', 'garden')

In [4]:
e = home.entities_by_device_id('548f06f0d7eb7b79de037861657e3e19')
e[0]['entity_id'], e[0]['name']

('binary_sensor.garden_security_status', None)

In [5]:

state = home.get_state('binary_sensor.garden_security_status')
state['state'], state.get('attributes')

('on',
 {'malware': 'safe',
  'network': 'safe',
  'securitySetting': 'safe',
  'systemCheck': 'safe',
  'update': 'outOfDate',
  'userInfo': 'safe',
  'attribution': 'Data provided by Synology',
  'device_class': 'safety',
  'friendly_name': 'garden Security status'})

## Model

In [7]:
response = model.complete("You are a helpful assistant", "What is the capital of France?")
response

' The capital city of France is Paris.'

## Area Summary

Attempt to get the list of devices in an area, the states of some of their entities, and create a summary.

In [7]:

def entity_data_for_area(area_id: str) -> list[str]
    entity_data = []
    for device in home.devices_by_area(area_id):
        entities = home.entities_by_device_id(device['id'])
        if not entities:
            continue
        #print(device['name'])
        for entity in entities:
            state = home.get_state(entity['entity_id'])
            attributes = state['attributes']
            friendly_name = attributes.get('friendly_name')
            entity_data.append(f"{friendly_name}: {state['state']}")
    return entity_data


In [11]:
PROMPT = f"""
You are an agent running in Home Assistant. Your job is to summarize the status of an area
which will be fed as input into other agents. The user will feed in details about
areas and devices in the home, and you will respond with a summary of the status of the area.

Your summaries are succint, and do not mention boring details or things that seem
very mundane or minor. A one sentence summary is best.


Here is an example of the input and output:

Area: Driveway
  - Tesla: Charging
  - Tesla Battery: 90%
  - Tesla Estimated range: 200 miles
  - Pedestrian Gate: No motion
  - Sprinkler: Off

Summary: The car is almost charged.
"""


In [12]:
entity_text = "\n".join([f"  - {data}" for data in entity_data])
message = f"""Area: Closet
{entity_text}
"""

response = model.complete(PROMPT, message)
response

' The garden is secure and the used space is at 12.6%.'